# DocTable Examples: Select
Here I show how to select data from a DocTable. We cover object-oriented conditional selects emulating the `WHERE` SQL clause, as well as some reduce functions.

In [1]:
import random
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
import doctable

In [2]:
import dataclasses
@dataclasses.dataclass
class Record(doctable.DocTableSchema):
    id: int = doctable.IDCol()
    name: str = doctable.Col(nullable=False)
    age: int = None
    is_old: bool = None

db = doctable.DocTable(target=':memory:', schema=Record, verbose=True)
print(db)

<DocTable (4 cols)::sqlite:///:memory::_documents_>


In [3]:
N = 10
for i in range(N):
    age = random.random() # number in [0,1]
    is_old = age > 0.5
    db.insert({'name':'user_'+str(i), 'age':age, 'is_old':is_old}, verbose=False)
print(db)

<DocTable (4 cols)::sqlite:///:memory::_documents_>


## Regular Selects
These functions all return lists of ResultProxy objects. As such, they can be accessed using numerical indices or keyword indices. For instance, if one select output row is ```row=(1, 'user_0')``` (after selecting "id" and "user"), it can be accessed such that ```row[0]==row['id']``` and ```row[1]==row['user']```.

In [4]:
# the limit argument means the result will only return some rows.
# I'll use it for convenience in these examples.
# this selects all rows
db.select(limit=2)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_
 LIMIT ? OFFSET ?


[Record(id=1, name='user_0', age=0.6643892612068403, is_old=True),
 Record(id=2, name='user_1', age=0.9053775349985783, is_old=True)]

In [5]:
db.select(['id','name'], limit=1)

DocTable: SELECT _documents_.id, _documents_.name 
FROM _documents_
 LIMIT ? OFFSET ?


[Record(id=1, name='user_0', age=None, is_old=None)]

In [6]:
# can also select by accessing the column object (db['id']) itself
# this will be useful later with more complex queries
db.select([db['id'],db['name']], limit=1)

DocTable: SELECT _documents_.id, _documents_.name 
FROM _documents_
 LIMIT ? OFFSET ?


[Record(id=1, name='user_0', age=None, is_old=None)]

In [7]:
db.select_first()

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_
 LIMIT ? OFFSET ?


Record(id=1, name='user_0', age=0.6643892612068403, is_old=True)

In [8]:
db.select('name',limit=5)

DocTable: SELECT _documents_.name 
FROM _documents_
 LIMIT ? OFFSET ?


['user_0', 'user_1', 'user_2', 'user_3', 'user_4']

In [9]:
db.select_first('age')

DocTable: SELECT _documents_.age 
FROM _documents_
 LIMIT ? OFFSET ?


0.6643892612068403

## Conditional Selects

In [10]:
db.select(where=db['id']==2)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ 
WHERE _documents_.id = ?


[Record(id=2, name='user_1', age=0.9053775349985783, is_old=True)]

In [11]:
db.select(where=db['id']<3)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ 
WHERE _documents_.id < ?


[Record(id=1, name='user_0', age=0.6643892612068403, is_old=True),
 Record(id=2, name='user_1', age=0.9053775349985783, is_old=True)]

In [12]:
# mod operator works too
db.select(where=(db['id']%2)==0, limit=2)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ 
WHERE _documents_.id % ? = ?
 LIMIT ? OFFSET ?


[Record(id=2, name='user_1', age=0.9053775349985783, is_old=True),
 Record(id=4, name='user_3', age=0.577166608655992, is_old=True)]

In [13]:
# note parantheses to handle order of ops with overloaded bitwise ops
db.select(where= (db['id']>=2) & (db['id']<=4) & (db['name']!='user_2'))

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ 
WHERE _documents_.id >= ? AND _documents_.id <= ? AND _documents_.name != ?


[Record(id=2, name='user_1', age=0.9053775349985783, is_old=True),
 Record(id=4, name='user_3', age=0.577166608655992, is_old=True)]

In [14]:
db.select(where=db['name'].in_(('user_2','user_3')))

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ 
WHERE _documents_.name IN (?, ?)


[Record(id=3, name='user_2', age=0.762910032919838, is_old=True),
 Record(id=4, name='user_3', age=0.577166608655992, is_old=True)]

In [15]:
db.select(where=db['id'].between(2,4))

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ 
WHERE _documents_.id BETWEEN ? AND ?


[Record(id=2, name='user_1', age=0.9053775349985783, is_old=True),
 Record(id=3, name='user_2', age=0.762910032919838, is_old=True),
 Record(id=4, name='user_3', age=0.577166608655992, is_old=True)]

In [16]:
# use of logical not operator "~"
db.select(where= ~(db['name'].in_(('user_2','user_3'))) & (db['id'] < 4))

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ 
WHERE _documents_.name NOT IN (?, ?) AND _documents_.id < ?


[Record(id=1, name='user_0', age=0.6643892612068403, is_old=True),
 Record(id=2, name='user_1', age=0.9053775349985783, is_old=True)]

In [17]:
# more verbose operators .and_, .or_, and .not_ are bound to the doctable package
db.select(where= doctable.or_(doctable.not_(db['id']==4)) & (db['id'] <= 2))

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ 
WHERE _documents_.id != ? AND _documents_.id <= ?


[Record(id=1, name='user_0', age=0.6643892612068403, is_old=True),
 Record(id=2, name='user_1', age=0.9053775349985783, is_old=True)]

In [18]:
# now with simple computation
ages = db.select(db['age'])
mean_age = sum(ages)/len(ages)
db.select(db['name'], where=db['age']>mean_age, limit=2)

DocTable: SELECT _documents_.age 
FROM _documents_
DocTable: SELECT _documents_.name 
FROM _documents_ 
WHERE _documents_.age > ?
 LIMIT ? OFFSET ?


['user_0', 'user_1']

In [19]:
# apply .label() method to columns
dict(db.select_first([db['age'].label('myage'), db['name'].label('myname')], as_dataclass=False))

DocTable: SELECT _documents_.age AS myage, _documents_.name AS myname 
FROM _documents_
 LIMIT ? OFFSET ?


{'myage': 0.6643892612068403, 'myname': 'user_0'}

## Column Operators
I bind the .min, .max, .count, .sum, and .mode methods to the column objects. Additionally, I move the .count method to a separate DocTable2 method.

In [20]:
db.select_first([db['age'].sum, db['age'].count, db['age']], as_dataclass=False)

DocTable: SELECT sum(_documents_.age) AS sum_1, count(_documents_.age) AS count_1, _documents_.age 
FROM _documents_
 LIMIT ? OFFSET ?


(6.553875813756212, 10, 0.6643892612068403)

In [21]:
# with labels now
dict(db.select_first([db['age'].sum.label('sum'), db['age'].count.label('ct')], as_dataclass=False))

DocTable: SELECT sum(_documents_.age) AS sum, count(_documents_.age) AS ct 
FROM _documents_
 LIMIT ? OFFSET ?


{'sum': 6.553875813756212, 'ct': 10}

## ORDER BY, GROUP BY, LIMIT
These additional arguments have also been provided.

In [22]:
# the limit is obvious - it has been used throughout these examples
db.select(limit=2)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_
 LIMIT ? OFFSET ?


[Record(id=1, name='user_0', age=0.6643892612068403, is_old=True),
 Record(id=2, name='user_1', age=0.9053775349985783, is_old=True)]

In [23]:
# orderby clause
db.select(orderby=db['age'].desc(), limit=2)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ ORDER BY _documents_.age DESC
 LIMIT ? OFFSET ?


[Record(id=10, name='user_9', age=0.9390342979922199, is_old=True),
 Record(id=2, name='user_1', age=0.9053775349985783, is_old=True)]

In [24]:
# compound orderby
db.select(orderby=(db['age'].desc(),db['is_old'].asc()), limit=2)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ ORDER BY _documents_.age DESC, _documents_.is_old ASC
 LIMIT ? OFFSET ?


[Record(id=10, name='user_9', age=0.9390342979922199, is_old=True),
 Record(id=2, name='user_1', age=0.9053775349985783, is_old=True)]

In [25]:
# can also use column name directly
# can only use ascending and can use only one col
db.select(orderby='age', limit=2)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ ORDER BY _documents_.age
 LIMIT ? OFFSET ?


[Record(id=5, name='user_4', age=0.30235086526336663, is_old=False),
 Record(id=6, name='user_5', age=0.32540985118674415, is_old=False)]

In [26]:
# groupby clause
# returns first row of each group without any aggregation functions
db.select(groupby=db['is_old'])

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ GROUP BY _documents_.is_old


[Record(id=5, name='user_4', age=0.30235086526336663, is_old=False),
 Record(id=1, name='user_0', age=0.6643892612068403, is_old=True)]

In [27]:
# compound groupby (weird example bc name is unique - have only one cat var in this demo)
db.select(groupby=(db['is_old'],db['name']), limit=3)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ GROUP BY _documents_.is_old, _documents_.name
 LIMIT ? OFFSET ?


[Record(id=5, name='user_4', age=0.30235086526336663, is_old=False),
 Record(id=6, name='user_5', age=0.32540985118674415, is_old=False),
 Record(id=1, name='user_0', age=0.6643892612068403, is_old=True)]

In [28]:
# groupby clause using max aggregation function
# gets match age for both old and young groups
db.select(db['age'].max, groupby=db['is_old'])

DocTable: SELECT max(_documents_.age) AS max_1 
FROM _documents_ GROUP BY _documents_.is_old


[0.32540985118674415, 0.9390342979922199]

## SQL String Commands and Additional Clauses
For cases where DocTable2 does not provide a convenient interface, you may submit raw SQL commands. These may be a bit more unwieldly, but they offer maximum flexibility. They may be used either as simply an addition to the WHERE or arbitrary end clauses, or accessed in totality.

In [29]:
qstr = 'SELECT age,name FROM {} WHERE id=="{}"'.format(db.tabname, 1)
results = db.execute(qstr)
dict(list(results)[0])

DocTable: SELECT age,name FROM _documents_ WHERE id=="1"


{'age': 0.6643892612068403, 'name': 'user_0'}

In [30]:
wherestr = 'is_old=="{}"'.format('1')
db.select(wherestr=wherestr, limit=2)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ 
WHERE (is_old=="1")
 LIMIT ? OFFSET ?


[Record(id=1, name='user_0', age=0.6643892612068403, is_old=True),
 Record(id=2, name='user_1', age=0.9053775349985783, is_old=True)]

In [31]:
# combine whrstr with structured query where clause
wherestr = 'is_old=="{}"'.format('1')
db.select(where=db['id']<=5, wherestr=wherestr)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ 
WHERE _documents_.id <= ? AND (is_old=="1")


[Record(id=1, name='user_0', age=0.6643892612068403, is_old=True),
 Record(id=2, name='user_1', age=0.9053775349985783, is_old=True),
 Record(id=3, name='user_2', age=0.762910032919838, is_old=True),
 Record(id=4, name='user_3', age=0.577166608655992, is_old=True)]

In [32]:
# combine whrstr with structured query where clause
wherestr = 'is_old=="{}"'.format('1')
db.select(where=db['id']<=5, wherestr=wherestr)

DocTable: SELECT _documents_.id, _documents_.name, _documents_.age, _documents_.is_old 
FROM _documents_ 
WHERE _documents_.id <= ? AND (is_old=="1")


[Record(id=1, name='user_0', age=0.6643892612068403, is_old=True),
 Record(id=2, name='user_1', age=0.9053775349985783, is_old=True),
 Record(id=3, name='user_2', age=0.762910032919838, is_old=True),
 Record(id=4, name='user_3', age=0.577166608655992, is_old=True)]

## Count Method and Get Next ID
```.count()``` is a convenience method. Mostly the same could be accomplished by ```db.select_first(db['id'].count)```, but this requires no reference to a specific column.

```.next_id()``` is especially useful if one hopes to enter the id (or any primary key column) into new rows manually. Especially useful because SQL engines don't provide new ids except when a single insert is performed.

In [33]:
db.count()

DocTable: SELECT count() AS count_1 
FROM _documents_
 LIMIT ? OFFSET ?


10

In [34]:
db.count(db['age'] < 0.5)

DocTable: SELECT count() AS count_1 
FROM _documents_ 
WHERE _documents_.age < ?
 LIMIT ? OFFSET ?


2

## Select as Pandas Series and DataFrame
These are especially useful when working with metadata because Pandas provides robust descriptive and plotting features than SQL alone. Good for generating sample information.

In [35]:
# must provide only a single column
db.select_series(db['age']).head(2)

DocTable: SELECT _documents_.age 
FROM _documents_


0    0.664389
1    0.905378
dtype: float64

In [36]:
db.select_series(db['age']).quantile([0.025, 0.985])

DocTable: SELECT _documents_.age 
FROM _documents_


0.025    0.307539
0.985    0.934491
dtype: float64

In [37]:
db.select_df(['id','age']).head(2)

DocTable: SELECT _documents_.id, _documents_.age 
FROM _documents_


,id,age
0,1,0.664389
1,2,0.905378


In [38]:
db.select_df('age').head(2)

DocTable: SELECT _documents_.age 
FROM _documents_


,age
0,0.664389
1,0.905378


In [39]:
# must provide list of cols (even for one col)
db.select_df([db['id'],db['age']]).corr()

DocTable: SELECT _documents_.id, _documents_.age 
FROM _documents_


,id,age
id,1.000000,0.092846
age,0.092846,1.000000


In [40]:
db.select_df([db['id'],db['age']]).describe().T

DocTable: SELECT _documents_.id, _documents_.age 
FROM _documents_


,count,mean,std,min,25%,50%,75%,max
id,10.0,5.500000,3.02765,1.000000,3.250000,5.500000,7.750000,10.000000
age,10.0,0.655388,0.21455,0.302351,0.587977,0.664459,0.784954,0.939034


In [41]:
mean_age = db.select_series(db['age']).mean()
df = db.select_df([db['id'],db['age']])
df['old_grp'] = df['age'] > mean_age
df.groupby('old_grp').describe()

DocTable: SELECT _documents_.age 
FROM _documents_
DocTable: SELECT _documents_.id, _documents_.age 
FROM _documents_


id                                              age            \
        count mean       std  min   25%  50%   75%   max count      mean   
old_grp                                                                    
False     4.0  5.5  1.290994  4.0  4.75  5.5  6.25   7.0   4.0  0.456334   
True      6.0  5.5  3.937004  1.0  2.25  5.5  8.75  10.0   6.0  0.788090   

                                                                     
              std       min       25%       50%       75%       max  
old_grp                                                              
False    0.165703  0.302351  0.319645  0.451288  0.587977  0.620407  
True     0.116414  0.664389  0.689124  0.777606  0.877109  0.939034

In [42]:
# more complicated groupby aggregation.
# calculates the variance both for entries above and below average age
mean_age = db.select_series(db['age']).mean()
df = db.select_df([db['name'],db['age']])
df['old_grp'] = df['age']>mean_age
df.groupby('old_grp').agg(**{
    'first_name':pd.NamedAgg(column='name', aggfunc='first'),
    'var_age':pd.NamedAgg(column='age', aggfunc=np.var),
})

DocTable: SELECT _documents_.age 
FROM _documents_
DocTable: SELECT _documents_.name, _documents_.age 
FROM _documents_


,first_name,var_age
old_grp,,
False,user_3,0.027457
True,user_0,0.013552


# Select with Buffer
In cases where you have many rows or each row contains a lot of data, you may want to perform a select query which makes requests in chunks. This is performed using the SQL OFFSET command, and querying up to buffsize while yielding each returned row. This system is designed this way because the underlying sql engine buffers all rows retreived from a query, and thus there is no way to stream data into memory without this system.

NOTE: The limit keyword is incompatible with this method - it will return all results. A workaround is to use the approx_max_rows param, which will return at minimum this number of rows, at max the specified number of rows plus buffsize.

In [43]:
for row_chunk in db.select_chunks(chunksize=2, where=(db['id']%2)==0, verbose=False):
    print(row_chunk)

[Record(id=2, name='user_1', age=0.9053775349985783, is_old=True), Record(id=4, name='user_3', age=0.577166608655992, is_old=True)]
[Record(id=6, name='user_5', age=0.32540985118674415, is_old=False), Record(id=8, name='user_7', age=0.7923015153822922, is_old=True)]
[Record(id=10, name='user_9', age=0.9390342979922199, is_old=True)]
[]
